In [1]:
import mlflow

In [2]:
# Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-13-127-25-124.ap-south-1.compute.amazonaws.com:5000/")

In [3]:
# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

<Experiment: artifact_location='s3://youtube-insights-chrome-plugin-mlflow-bucket/547399241317235994', creation_time=1750433363378, experiment_id='547399241317235994', last_update_time=1750433363378, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna

c:\ProgramData\anaconda3\envs\yt-insights-ch-plugin\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [6]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for SVM

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

In [9]:
# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

In [10]:
# Step 6: Optuna objective function for SVM
def objective_svm(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])

    model = SVC(C=C, kernel=kernel, random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))

In [11]:
# Step 7: Run Optuna for SVM, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_svm, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = SVC(C=best_params['C'], kernel=best_params['kernel'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "SVM"
    log_mlflow("SVM", best_model, X_train, X_test, y_train, y_test)

In [12]:
# Run the experiment for SVM
run_optuna_experiment()

[I 2025-06-20 21:18:42,619] A new study created in memory with name: no-name-87e5466f-3b9c-432d-8e9f-e1fa98f9c6ce
[I 2025-06-20 21:25:43,946] Trial 0 finished with value: 0.3993870217712957 and parameters: {'C': 0.0014893690781816883, 'kernel': 'rbf'}. Best is trial 0 with value: 0.3993870217712957.
[I 2025-06-20 21:32:09,624] Trial 1 finished with value: 0.6536673007820757 and parameters: {'C': 0.04963068022595884, 'kernel': 'rbf'}. Best is trial 1 with value: 0.6536673007820757.
[I 2025-06-20 21:38:04,394] Trial 2 finished with value: 0.4458888184316212 and parameters: {'C': 0.00021309214709520528, 'kernel': 'linear'}. Best is trial 1 with value: 0.6536673007820757.
[I 2025-06-20 21:43:22,789] Trial 3 finished with value: 0.7141196364404988 and parameters: {'C': 0.07883573611285008, 'kernel': 'rbf'}. Best is trial 3 with value: 0.7141196364404988.
[I 2025-06-20 21:49:53,793] Trial 4 finished with value: 0.7759458888184316 and parameters: {'C': 8.802236668245953, 'kernel': 'linear'}. 